In [1]:
import pandas as pd

In [37]:
df01 = pd.read_parquet('yellow_tripdata_2023-01.parquet')

### Q1

In [14]:
print(len(df01.columns))
print(df01.shape[1])

19
19


### Q2

In [23]:
df01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [24]:
duration = df01.iloc[0]['tpep_dropoff_datetime'] - df01.iloc[0]['tpep_pickup_datetime']
duration.total_seconds() / 60

8.433333333333334

In [38]:
df01['duration'] = df01['tpep_dropoff_datetime'] - df01['tpep_pickup_datetime']
df01.duration = df01.duration.apply(lambda td: td.total_seconds() / 60)

In [39]:
df01.duration.describe()['std']

42.594351241920904

### Q3

In [42]:
df01_outliers_removed = df01[ (df01['duration'] >=1) & (df01['duration']<=60) ]

In [44]:
len(df01_outliers_removed)/len(df01)

0.9812202822125979

### Q4

In [49]:
categorical = ['PULocationID', 'DOLocationID']

df01_outliers_removed[categorical] = df01_outliers_removed[categorical].astype(str)

/tmp/ipykernel_184/3033328298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df01_outliers_removed[categorical] = df01_outliers_removed[categorical].astype(str)


In [52]:
from sklearn.feature_extraction import DictVectorizer

train_dicts = df01_outliers_removed[categorical].to_dict(orient='records')

In [87]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [60]:
X_train.shape

(3009173, 515)

### Q5

In [64]:
from sklearn.linear_model import LinearRegression

y_train = df01_outliers_removed['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [68]:
from sklearn.metrics import mean_squared_error

y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 7.649261927665777


### Q6

In [91]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [92]:
df02 = read_dataframe('yellow_tripdata_2023-02.parquet')

In [93]:
val_dicts = df02[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [94]:
y_val = df02['duration'].values

y_pred = lr.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"Root Mean Squared Error on validation data: {rmse}")

Root Mean Squared Error on validation data: 7.811817548344513
